In [280]:
import win32com.client
import sys

In [281]:
# StateNode initial : 3
# StateNode final : 4
# StateNode History(shallow) : 5
# StateNode Synch: 6
# StateNode Junction: 10
# StateNode choice : 11
# StateNode Terminate : 12
# StateNode EntryPoint : 13
# StateNode ExitPoint : 14
# StateNode Historystar(deep) : 15
# Synchronization Horizontal : 0
# Synchronization Vertical : 1

packageSubtypes = {
    "Initial" :  3,
    "Final" :  4,
    "HistoryShallow" :  5,
    "Synch" : 6,
    "Junction" :  10,
    "choice" : 11,
    "Terminate" : 12,
    "EntryPoint" : 13,
    "ExitPoint" : 14,
    "HistoryDeep" : 15,
    "SyncHorizontal" : 0,
    "SyncVertical" :  1,
    "State" : 0,
    "StateMachine" : 0
}


In [282]:
class InnerTransition:
    def __init__(self):
        self.m_event = ""
        self.m_toState = ""
        self.m_guard = ""
        self.m_action = ""

    def getName(self):
        return self.m_toState
    
    def update(self, dictElem):
        if isinstance(dictElem, dict):
            [setattr(self, attrib, dictElem[attrib]) for attrib in dictElem.keys() if hasattr(self, attrib)]
                        

In [283]:
# unit test of Inner Transition
obj = InnerTransition()
dictElem = {"m_event" : 2 , "m_guard" : 3, "nayan" : 4}
obj.update(dictElem)
print(obj.m_event)

pass

2


In [284]:
class State:
    def __init__(self, name = "UNK_State"):
        self.m_name = name
        self.m_notes = ""
        self.m_entryAction = ""
        self.m_doAction =""
        self.m_exitAction = ""
        self.m_innerTransition = {}

    def getNameOfFromState(self):
        return self.m_name
    
    def updateNotes(self, notes):
        self.m_notes = notes
    
    def updateDoAction(self, action):
        self.m_doAction = action

    def updateEntryAction(self, action):
        self.m_entryAction = action

    def updateExitAction(self, action):
        self.m_exitAction = action

    def addInnerTransition(self, innerTransition):
        # Transition is of Type Inner Transition
        if isinstance(innerTransition, dict) :
            if innerTransition.get("m_event") != None:
                self.m_innerTransition[innerTransition["m_event"]] = InnerTransition()
                self.m_innerTransition[innerTransition["m_event"]].update(innerTransition)
    
    def checkIfInnerTransitionExists(self, name):
        if self.m_innerTransition.get(name) == None:
            return False
        else:
            return True
    
    def updateInnerTransition(self, dictElem):
        if isinstance(dictElem, dict) :
            if dictElem.get("m_event") != None and dictElem.get("m_toState") != None:
                if self.checkIfInnerTransitionExists(dictElem["m_toState"]):
                    self.m_innerTransition[dictElem["m_event"]].update(dictElem)
                else:
                    self.addInnerTransition(dictElem)
            else:
                return False  
        else:
            return False
                

        
        

In [285]:
state = State("N")

val1 = {"m_event" : "Y" , "m_guard" : 3, "m_action" : 4, "m_toState" : "X"}
val2 = {"m_event" : "" , "m_guard" : 3, "m_action" : 4, "m_toState" : "X"}

state.updateInnerTransition(val1)
state.updateInnerTransition(val2)

pass

In [286]:
class OuterTransition:
    def __init__(self):
        self.m_fromState = {}
        
    def addOuterState(self, stateName):
        self.m_fromState[stateName] = State(stateName)

    def addDoAction(self, stateName, doACtion):
        if self.isStateExisting(stateName):
            self.m_fromState[stateName].updateDoAction(doACtion)

    def addEntryAction(self, stateName, entryAction):
        if self.isStateExisting(stateName):
            self.m_fromState[stateName].updateEntryAction(entryAction)
            
    def addExitAction(self, stateName, exitAction):
        if self.isStateExisting(stateName):
            self.m_fromState[stateName].updateEntryAction(exitAction)

    def addNotes(self, stateName, notes):
        if self.isStateExisting(stateName):
            self.m_fromState[stateName].updateNotes(notes)

    def addInnerTransition(self, stateName, innerTransition):
        # trainstion is the object of class innerTransition
        if self.m_fromState.get(stateName) != None:
            ret = self.m_fromState[stateName].updateInnerTransition(innerTransition)
            return ret
        else:
            return False

    def isStateExisting(self, stateName):
        if self.m_fromState.get(stateName) == None:
            return False
        else:
            return True

    

In [287]:
class Container:
    def __init__(self):
        self.m_containerOuter = OuterTransition()
        self.m_unusedTransition = [] # tuple of unsed transitions for debugging

    def getOuterContainerObject(self):
        return self.m_containerOuter

    # def addOuterTransition(self, outerTransitions):
    #     self.m_containerOuter.add(outerTransitions)


In [288]:
class EABasicSetup:
    def __init__(self):
        self.m_repository = ""
        self.m_model = ""
        self.m_package = ""
        self.m_root = ""
        self.m_modelCount = 0

    def setRepository(self, repo):
        if repo.ConnectionString == '':
            # todo : implement central error handler
            # exceptions to be handled by central error handler
            sys.stderr.write( "EA has no Model loaded\n" )
            return False
        else:
            print ("connecting to...", repo.ConnectionString)
            self.m_repository = repo
            return True
        
    def addModel(self, modelName):
        # This is the top level entry point to an Enterprise Architect project file
        self.m_model = self.m_repository.Models.AddNew(modelName, "EAPackage")
        self.m_model.Update()
        self.m_repository.Models.Refresh()
        self.m_modelCount = self.m_repository.Models.Count

    def addPackage(self, packageName):
        self.m_package = self.m_model.Packages.AddNew(packageName, "Simple")
        self.m_package.Update()

    def setRoot(self):
        if self.m_modelCount >= 1:
            self.m_root = self.m_repository.Models.GetAt(0)

    def deleteAllModelsExceptLatest(self):
        # Delete all models except the last one
        for i in range(0, self.m_modelCount- 1 , 1):
            # uncomment below lines to know the model being deleted
            # model = m_Repository.Models.GetAt(i)
            self.m_repository.Models.DeleteAt(0, True)
            self.m_repository.Models.Refresh()

    def reloadDiagram(self, id):
        self.m_repository.ReloadDiagram(id)

    def getPackage(self):
        return self.m_package
    

    def getRepository(self):
        return self.m_repository
   


In [289]:
class EA:
    def __init__(self):
        self.m_eaApp = ""
        self.m_eaAppRepo = ""
        self.m_eaSetup = EABasicSetup()
        self.m_modelName = "StateMachine"
        self.m_packageName = "StateMachinePackage"

    def setApplicationLink(self):
        try:
            self.m_eaApp = win32com.client.Dispatch("EA.App")
        except:
            sys.stderr.write( "Unable to connect to EA\n" )
            return False
        return True

    def makeInitialSetup(self):
        self.m_eaAppRepo = self.m_eaApp.Repository
        check = self.m_eaSetup.setRepository(self.m_eaAppRepo)

        if check == True:
            self.m_eaSetup.addModel(self.m_modelName)
            self.m_eaSetup.addPackage(self.m_packageName)
            self.m_eaSetup.deleteAllModelsExceptLatest()
            self.m_eaSetup.setRoot()
            return True
        else:
            return False
        
    def reloadProject(self):
        self.m_eaApp.Project.ReloadProject()

    def getSetup(self):
        return self.m_eaSetup

In [290]:
class ModelData:
    def __init__(self):
        self.m_widget = []
        self.m_child = [] 

In [291]:
class Elements:
    def __init__(self):
        self.m_elements = {}
        self.m_states = {}
        self.m_stateMachines = {}
        self.m_stateNodes = {}
        self.m_repository = ""
        self.m_packageName = ""

    def connectRootPackage(self, root):
        self.m_packageName  = root.getPackage()
        self.m_repository = root.getRepository()
    
    def addMainElements(self, elementName, type, subtype, entryAction = "", doACtion = "", exitAction = ""):
        if (elementName != "") and (type == "StateMachine" or type == "StateNode") and (subtype in packageSubtypes):
            elem = self.m_packageName.Elements.AddNew(elementName, type)
            elem.Subtype = packageSubtypes[subtype]
            elem.Update()
            self.m_repository.Models.Refresh()
            self.addToDatabase(elementName, elem, type)

        if (elementName != "") and (type == "State") and (subtype in packageSubtypes):
            elem = self.m_packageName.Elements.AddNew(elementName, type)
            elem.Subtype = packageSubtypes[subtype]
            elem.Methods.AddNew(entryAction, "entry").Update()
            elem.Methods.AddNew(doACtion, "do").Update()
            elem.Methods.AddNew(exitAction, "exit").Update()
            elem.Update()
            self.m_repository.Models.Refresh()
            self.addToDatabase(elementName, elem, type)

            

    def addSubStates(self, stateMachineName, elementName, type, subtype):
        if (stateMachineName in self.m_stateMachines) and (elementName != "") and (type == "State" or type == "StateMachine" or type == "StateNode") and (subtype in packageSubtypes):
            elem = self.m_stateMachines[stateMachineName].Elements.AddNew(elementName, type)
            elem.Subtype = packageSubtypes[subtype]
            self.m_stateMachines[stateMachineName].Update()
            self.m_repository.Models.Refresh()
            self.addToDatabase(elementName, elem, type)
        


    def addToDatabase(self, elementName, elem, type):
        self.m_elements[elementName] = elem

        if type == "State" :
            self.m_states[elementName] = elem
            
        elif type == "StateMachine":
            self.m_stateMachines[elementName] = elem

        elif type == "StateNode":
            self.m_stateNodes[elementName] = elem


    def getElements(self):
        return self.m_elements



In [292]:
class Diagrams:
    def __init__(self):
        self.m_diagramPackage = ""
        self.m_diagrams = {}

    def createDiagramObject(self, root):
        m_packageName  = root.getPackage()
        self.m_diagramPackage = m_packageName.Diagrams.AddNew( "State Diagram", "Statechart" )
        self.m_diagramPackage.Notes = "State Machine Diagram"
        self.m_diagramPackage.Update()

    def createInstancesOfAllElements(self, elements):
        for elem in elements:
            self.m_diagrams[elem] = self.m_diagramPackage.DiagramObjects.AddNew(elements[elem].ElementGUID, "")
            self.m_diagrams[elem].ElementID = elements[elem].ElementID
            self.m_diagrams[elem].Update()

In [293]:
class Transitions:
    def __init__(self) -> None:
        self.m_name = ""
        self.m_notes = ""
        self.m_sourceElement = ""
        self.m_destinationElement = ""
        self.m_connectors = {}

    def connectElements(self, root, elements, sourceElement, destElement):
        # get repo and elements
        m_RepositoryName  = root.getRepository()
        m_elements = elements.getElements()
        self.m_name = sourceElement + "to" + destElement
        # get source and destination element ID
        self.m_sourceElement = m_RepositoryName.GetElementByID(m_elements[sourceElement].ElementID)
        self.m_destinationElement = m_RepositoryName.GetElementByID(m_elements[destElement].ElementID)
        # create connection
        self.createConnection()

    def createConnection(self):
        # create a new connector and add
        connector = self.m_sourceElement.Connectors.AddNew("", "StateFlow")
        connector.SupplierID = self.m_destinationElement.ElementID
        connector.Update()
        # add connector to database
        self.m_connectors[self.m_name] = connector


    def addProperties(self, action = "", event = "", guard = ""):
        self.m_connectors[self.m_name].TransitionAction = action
        self.m_connectors[self.m_name].TransitionEvent = event
        self.m_connectors[self.m_name].TransitionGuard = guard
        self.m_connectors[self.m_name].Update()


    def getName(self):
        return self.m_name

In [294]:
class EAInterface:
    def __init__(self):
        self.m_eaRoot = ""
        self.m_Elements = Elements()
        self.m_diagrams = Diagrams()
        self.m_transitions = []
        pass

In [295]:
import template
import importlib
importlib.reload(template)
import re



def parse():
    dataDict = {
        "state" : [],
        "initial" : [],
        "entry" : [],
        "do" : [],
        "exit" : [],
        "transitions" : [],
        "CondTransitions" : [],
        "CondTransitionsWithEvents" : []
    }

    for data in template.textData:
        for key in template.patternRegEx.keys():
            matches = re.match(template.patternRegEx[key], data)
            if matches != None:
                groups = matches.groups()
                dictionary = {}
                # check if pattern groups are matching
                index = len(template.regExMatchGroups[key])
                if index == len(groups):
                    for i in range(index):
                        dictionary[template.regExMatchGroups[key][i]] = groups[i]
                    dataDict[key].append(dictionary)
                    break
                else:
                    pass
            else:
                pass

    return dataDict

In [296]:

#create EA object
ea = EA()

# Try to establish link with EA
ret = ea.setApplicationLink()

# check if initial link is completed
if ret == True:
    ea.makeInitialSetup()
else:
    exit()

# check if initial setup was completed 
if ret == True:
    ea.reloadProject()
else:
    exit()
    

eaSetup = ea.getSetup()
# create a normal container

# dataMain = [
#     ["Machine1", "StateMachine", "StateMachine"],
#     ["Initial", "StateNode", "Initial"],
#     ["State3", "State", "State"]
# ]

# dataSub = [
#     ["Machine1", "State1", "State", "State"],
#     ["Machine1", "State2", "State", "State"]
# ]


data = parse()

containerVal = Container()
contObj = containerVal.getOuterContainerObject()
fromStatesObjDict = contObj.m_fromState

for key in data.keys():
    if key == "state":
        for value in data[key]:
            contObj.addOuterState(value["m_fromState"])

    elif key == "entry":
        for value in data[key]:
            contObj.addEntryAction(value["m_fromState"], value["m_entryAction"])

    elif key == "do":
        for value in data[key]:
            contObj.addDoAction(value["m_fromState"], value["m_doAction"])

    elif key == "exit":
        for value in data[key]:
            contObj.addExitAction(value["m_fromState"], value["m_exitAction"])

    elif key == "transitions":
        for value in data[key]:
            contObj.addInnerTransition(value["m_fromState"], value)

    elif key == "CondTransitions":
        for value in data[key]:
            contObj.addInnerTransition(value["m_fromState"], value)

    elif key == "CondTransitionsWithEvents":
        for value in data[key]:
            contObj.addInnerTransition(value["m_fromState"], value)



# EA Model creation
eaInterface = EAInterface()
eaInterface.m_Elements.connectRootPackage(eaSetup)

# for val in dataMain:
#     eaInterface.m_Elements.addMainElements(val[0], val[1], val[2])

# for val in dataSub:
#     eaInterface.m_Elements.addSubStates(val[0], val[1], val[2], val[3])

# add state objects

for data in fromStatesObjDict.keys():
    eaInterface.m_Elements.addMainElements(fromStatesObjDict[data].m_name, "State", "State", fromStatesObjDict[data].m_entryAction, fromStatesObjDict[data].m_doAction, fromStatesObjDict[data].m_exitAction)


# add diagram object and create instances of elements
eaInterface.m_diagrams.createDiagramObject(eaSetup)
eaInterface.m_diagrams.createInstancesOfAllElements(eaInterface.m_Elements.getElements())

# create transitions
for state in fromStatesObjDict.keys():
    for innerTransition in fromStatesObjDict[state].m_innerTransition.keys():
        transition = Transitions()
        transition.connectElements(eaSetup, eaInterface.m_Elements, state, fromStatesObjDict[state].m_innerTransition[innerTransition].m_toState)
        transition.addProperties(fromStatesObjDict[state].m_innerTransition[innerTransition].m_action, fromStatesObjDict[state].m_innerTransition[innerTransition].m_event , fromStatesObjDict[state].m_innerTransition[innerTransition].m_guard)
        eaInterface.m_transitions.append(transition)



connecting to... C:\Users\nayan\education\MasterThesis\MachineLearning_NLP\11_XMI\try.eap
